<a href="https://colab.research.google.com/github/tomvangasse/game-store-manager/blob/main/tomsMenuProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Upload files here
Click run and then click choose files:

    Video_Game_Info.txt
    Board_Game_Info.txt
    Subscription_Info.txt
    Game_feedback.txt
    Booking.txt
    Rental.txt
    feedbackManager.pyc
    subscriptionManager.pyc
'''


from google.colab import files
uploaded = files.upload()

Saving Board_Game_Info.txt to Board_Game_Info.txt
Saving Booking.txt to Booking.txt
Saving Game_Feedback.txt to Game_Feedback.txt
Saving Rental.txt to Rental.txt
Saving Subscription_Info.txt to Subscription_Info.txt
Saving Video_Game_Info.txt to Video_Game_Info.txt
Saving feedbackManager.pyc to feedbackManager.pyc
Saving subscriptionManager.pyc to subscriptionManager.pyc


In [ ]:
import subscriptionManager as smTV
import feedbackManager as fbTV

sub = smTV.load_subscriptions()

print(sub)


Register / login

In [2]:
def checkSubscription(userID):
    while True:
        subscriptions = smTV.load_subscriptions()
        if smTV.check_subscription(userID, subscriptions):
            return userID
        else:
            login(userID)

    subType = subscriptions[userID]["SubscriptionType"]


def register(userID):
    import subscriptionManager as smTV
    import datetime
    from dateutil.relativedelta import relativedelta as rd

    print("Unregistered account, please register now:")
    while True:
        subscriptionType = input("Enter subscription type (Basic or premium): ").lower()
        if subscriptionType != "basic" and subscriptionType != "premium":
            print("Invalid subscription type")
        else:
            break
    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    endTime = (date + rd(years=3)).strftime("%Y-%m-%d")

    with open("Subscription_Info.txt", mode = "a") as file:
        file.write(f"\n{userID},{subscriptionType},{startTime},{endTime}")

    print(f"Logged in as {userID}")
    return subscriptionType


def login(userID):
    import subscriptionManager as smTV

    subscriptions = smTV.load_subscriptions()

    if smTV.check_subscription(userID, subscriptions):
        print(f"Logged in as {userID}")

    else:
        register(userID)

def checkUserLength(userID):
    if len(userID) != 4:
        return False
    else:
        return True

Game search

In [3]:
import csv

def getVideoDict():
    with open("Video_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def getBoardDict():
    with open("Board_Game_Info.txt", mode="r", newline="") as file:
        reader = csv.DictReader(file)
        boardGameDict = {row["GameID"]: row for row in reader}
    return boardGameDict

def printGames(gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        for i in boardDict:
            currentGame = boardDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    elif gameType == "v":
        for i in videoDict:
            currentGame = videoDict[i]
            print(f"{currentGame["GameID"]}: {(currentGame["Game name"]): <25} Available / Unavailable: {currentGame["Available copies"]} / {currentGame["Unavailable copies"]}")
        print("\n")
    else:
        return

def printGameInfo(GameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        gameInfo = boardDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    elif gameType == "v":
        gameInfo = videoDict[GameID]
        for key, value in gameInfo.items():
            print(f"{key: >20}:  {value}")
        print("\n")
    else:
        return


def gameSearch():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    gameType = input('''Board games (b)
Video games (v)
Quit (q)
''')

    if gameType == "q":
        return False

    gameSelection = input('''Print games (press enter)
Select a specific game (enter GameID)
''')
    gameSelection
    if gameSelection == "":
        printGames(gameType)
    elif gameSelection == "q":
        return False
    else:
        printGameInfo(gameSelection, gameType)

while True:
    if gameSearch() == False:
        break


Board games (b)
Video games (v)
Quit (q)
q


Update text files:

In [18]:
def updateVideo(videoDict):
    headers = ["GameID", "Game name", "Platform", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Video_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in videoDict.items():
        writer.writerow(value)
    return

def updateBoard(boardDict):
    headers = ["GameID", "Game name", "No of players", "Genre", "Date added", "Available copies", "Unavailable copies"]
    writer = csv.DictWriter(open("Board_Game_Info.txt", mode="w", newline=""), fieldnames=headers)
    writer.writeheader()
    for key, value in boardDict.items():
        writer.writerow(value)
    return

def updateRental(rentalList):
    with open("Rental.txt", mode="w", newline="") as file:
        file.write("Rental userID,GameID,Rental date,Return date\n")
        for row in rentalList:
            file.write(f"{','.join(row)}\n")
    return



Game rent

In [27]:
import subscriptionManager as smTV
import feedbackManager as fbTV


def updateBoardFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    boardDict = getBoardDict()
    availableCopies = boardDict[GameID]["Available copies"]
    unavailableCopies = boardDict[GameID]["Unavailable copies"]
    boardDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    boardDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateBoard(boardDict)

def updateVideoFromRentals(GameID, action):
    value = 0
    if action == "rent":
        value = 1
    elif action == "return":
        value = -1
    else:
        return

    videoDict = getVideoDict()
    availableCopies = videoDict[GameID]["Available copies"]
    unavailableCopies = videoDict[GameID]["Unavailable copies"]
    videoDict[GameID]["Available copies"] = str(int(availableCopies) - value)
    videoDict[GameID]["Unavailable copies"] = str(int(unavailableCopies) + value)
    updateVideo(videoDict)

def updateRentGame(gameID, userID):
    from dateutil.relativedelta import relativedelta as rd
    import datetime

    date = datetime.datetime.now()
    startTime = date.strftime("%Y-%m-%d")
    with open("Rental.txt", mode="a", newline="") as file:
        file.write(f"\n{userID},{gameID},{startTime},")


def checkAvailability(gameID, gameType):
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    if gameType == "b":
        currentGame = boardDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True

    elif gameType == "v":
        currentGame = videoDict[gameID]
        if currentGame["Available copies"] == "0":
            return False
        else:
            return True
    else:
        return


def rentGameMain():
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    while True:
        userID = input("Enter user ID: ")
        if checkUserLength(userID):
            userID = checkSubscription(userID)
            break
        else:
            print("Invalid user ID")


    selectedGameID = input('''Enter the GameID for the desired game:
''')
    if selectedGameID in videoDict:
        if checkAvailability(selectedGameID, "v"):
            updateVideoFromRentals(selectedGameID, "rent")
            updateRentGame(selectedGameID, userID)
            currentGame = videoDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    elif selectedGameID in boardDict:
        # print(checkAvailability(selectedGameID, "b"))
        if checkAvailability(selectedGameID, "b"):
            updateBoardFromRentals(selectedGameID, "rent")
            updateRentGame(selectedGameID, userID)
            currentGame = boardDict[selectedGameID]
            print(f"{currentGame["Game name"]} has been rented by {userID}")

        else:
            print("Game not available")

    else:
        print("Invalid GameID")



#with open("Rental.txt", mode="a") as file:
    #file.write(f"\n{gameID},{Rental date}

rentGameMain()

Enter user ID: tomv
Enter the GameID for the desired game:
mine01v
Minecraft has been rented by tomv


Returns


In [38]:
import datetime


def getRentalList():
    with open("Rental.txt", mode="r", newline="") as file:
        count = 0
        rentalList = []
        for line in file:
            if count == 0:
                count += 1
                continue
            else:
                currentLine = line.rstrip().split(",")
            count += 1
            rentalList.append(currentLine)
    return rentalList

def getUserRental(userID, rentalList):
    userRentals = []
    for row in rentalList:
        if row[0] == userID and row[3] == "":
            userRentals.append(row)
    return userRentals

def printRentedGames(userRentals):
    print("\nRented games:")
    videoDict = getVideoDict()
    boardDict = getBoardDict()
    for row in userRentals:
        gameID = row[1]
        if gameID in videoDict:
            print(f"{gameID}:  {videoDict[gameID]['Game name']}")
        elif gameID in boardDict:
            print(f"{gameID}:  {boardDict[gameID]['Game name']}")
        else:
            print("Invalid GameID")

def updateReturnList(rentalList, userID, gameID):
    for row in rentalList:
        if row[0] == userID and row[1] == gameID and row[3] == '':
            row[3] = datetime.datetime.now().strftime("%Y-%m-%d")
            break
    return rentalList


def returnGameMain():
    videoDict = getVideoDict()
    boardDict = getBoardDict()

    date = datetime.datetime.now()
    rentalList = getRentalList()
    while True:
        userID = input("Enter user ID: ")
        if checkUserLength(userID):
            userID = checkSubscription(userID)
            break
        else:
            print("Invalid user ID")
    userRental = getUserRental(userID, rentalList)
    printRentedGames(userRental)

    while True:
        selectedGameID = input("Enter selected gameID: ")
        for row in userRental:
            if row[1] == selectedGameID:
                break
        else:
            print("Invalid GameID")
            continue
        break


    rentalList = updateReturnList(rentalList, userID, selectedGameID)
    print(rentalList)
    if selectedGameID in videoDict:
        updateVideoFromRentals(selectedGameID, "return")
    elif selectedGameID in boardDict:
        updateBoardFromRentals(selectedGameID, "return")
    updateRental(rentalList)

    print("Game returned")

returnGameMain()


Enter user ID: tomv

Rented games:
btls01b:  Battleship
mine01v:  Minecraft
Enter selected gameID: mine01v
[['abcd', 'tloz01', '2025-10-10', '25-11-10'], ['tomv', 'btls01b', '2025-10-20', ''], ['tomv', 'mine01v', '2025-10-20', '2025-11-20'], ['tomv', 'fzhz01v', '2025-10-20', '2025-11-20'], ['tomv', 'mine01v', '2025-11-20', '2025-11-20']]
Game returned
